In [4]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

print("Libraries imported")

Libraries imported


## Part 1: Fetch the Postal Codes and Boroughs / Neighborhoods

In [5]:
boroughs_df = pd.read_excel("London_Boroughs.xlsx")
boroughs_df

,Code,Neighborhood
0,E09000001,City of London
1,E09000002,Barking and Dagenham
2,E09000003,Barnet
3,E09000004,Bexley
4,E09000005,Brent
5,E09000006,Bromley
6,E09000007,Camden
7,E09000008,Croydon
8,E09000009,Ealing
9,E09000010,Enfield


## Part 2: Fetch the latitude and longitude of the city boroughs

In [6]:
from geopy.geocoders import Nominatim

In [7]:
address = 'London, UK'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
city_latitude = location.latitude
city_longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(city_latitude, city_longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [8]:
latitudes = []
longitudes = []

for index, row in boroughs_df.iterrows():
    borough = row['Neighborhood']

    address = '{}, London, UK'.format(borough)

    location = geolocator.geocode(address)

    latitude = location.latitude
    longitude = location.longitude

    latitudes.append(latitude)
    longitudes.append(longitude)

boroughs_df['Latitude'] = latitudes
boroughs_df['Longitude'] = longitudes


In [9]:
boroughs_df

,Code,Neighborhood,Latitude,Longitude
0,E09000001,City of London,51.515618,-0.091998
1,E09000002,Barking and Dagenham,51.554091,0.150489
2,E09000003,Barnet,51.653090,-0.200226
3,E09000004,Bexley,51.441679,0.150488
4,E09000005,Brent,51.563996,-0.275906
5,E09000006,Bromley,51.402805,0.014814
6,E09000007,Camden,51.542797,-0.162480
7,E09000008,Croydon,51.371305,-0.101957
8,E09000009,Ealing,51.512655,-0.305195
9,E09000010,Enfield,51.652085,-0.081018


## Part 3: Explore and Cluster city boroughs/neighborhoods

## 3.1. Visualize city neighborhoods

In [10]:
address = 'London, UK'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
city_latitude = location.latitude
city_longitude = location.longitude

In [11]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [12]:
# create map of New York using latitude and longitude values
map_city = folium.Map(location=[city_latitude, city_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(boroughs_df['Latitude'], boroughs_df['Longitude'], boroughs_df['Neighborhood']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city)  
    
map_city

## 3.2 Explore City Neighborhoods

In [13]:
CLIENT_ID = 'G3I1S3STTCK0OMQXUGZPMWJZ2LAZN0AUYT0NB23OBVG1TJEU' # your Foursquare ID
CLIENT_SECRET = 'NOOBRTQ22EA3OPFVBE5ULOXSXX3MTMHKDQLWLQMT3M0ACUXR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G3I1S3STTCK0OMQXUGZPMWJZ2LAZN0AUYT0NB23OBVG1TJEU
CLIENT_SECRET:NOOBRTQ22EA3OPFVBE5ULOXSXX3MTMHKDQLWLQMT3M0ACUXR


In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create a function to explore all the neighborhoods in Toronto

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        print('{}: {} venues'.format(name, len(results)))

        
        # return only relevant information for each nearby venue

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
neighborhood_names = boroughs_df.loc[:, 'Neighborhood'] # neighborhood names
neighborhood_latitudes = boroughs_df.loc[:, 'Latitude'] # neighborhood latitude values
neighborhood_longitudes = boroughs_df.loc[:, 'Longitude'] # neighborhood longitude values

boroughs_venues = getNearbyVenues(neighborhood_names, neighborhood_latitudes, neighborhood_longitudes)

boroughs_venues.head()

City of London: 100 venues
Barking and Dagenham: 14 venues
Barnet: 26 venues
Bexley: 9 venues
Brent: 68 venues
Bromley: 62 venues
Camden: 100 venues
Croydon: 88 venues
Ealing: 100 venues
Enfield: 43 venues
Greenwich: 100 venues
Hackney: 100 venues
Hammersmith and Fulham: 100 venues
Haringey: 45 venues
Harrow: 22 venues
Havering: 100 venues
Hillingdon: 11 venues
Hounslow: 67 venues
Islington: 100 venues
Kensington and Chelsea: 100 venues
Kingston upon Thames: 100 venues
Lambeth: 100 venues
Lewisham: 31 venues
Merton: 67 venues
Newham: 30 venues
Redbridge: 10 venues
Richmond upon Thames: 25 venues
Southwark: 100 venues
Sutton: 7 venues
Tower Hamlets: 100 venues
Waltham Forest: 30 venues
Wandsworth: 77 venues
Westminster: 100 venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,City of London,51.515618,-0.091998,Goodman Steak House Restaurant,51.514398,-0.090745,Steakhouse
1,City of London,51.515618,-0.091998,The Ned Hotel,51.513755,-0.090067,Hotel
2,City of London,51.515618,-0.091998,Hawksmoor Guildhall,51.515498,-0.090849,Steakhouse
3,City of London,51.515618,-0.091998,City Càphê,51.514750,-0.091545,Vietnamese Restaurant
4,City of London,51.515618,-0.091998,Daunt Books,51.513982,-0.092995,Bookstore


## 3.3 Analyze neighborhoods Venues

In [17]:
print(boroughs_venues.shape)
print(len(boroughs_venues['Neighborhood'].unique()))
boroughs_venues.head()

(2132, 7)
33


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,City of London,51.515618,-0.091998,Goodman Steak House Restaurant,51.514398,-0.090745,Steakhouse
1,City of London,51.515618,-0.091998,The Ned Hotel,51.513755,-0.090067,Hotel
2,City of London,51.515618,-0.091998,Hawksmoor Guildhall,51.515498,-0.090849,Steakhouse
3,City of London,51.515618,-0.091998,City Càphê,51.514750,-0.091545,Vietnamese Restaurant
4,City of London,51.515618,-0.091998,Daunt Books,51.513982,-0.092995,Bookstore


In [18]:
boroughs_venues.shape

(2132, 7)

In [19]:
boroughs_venues.groupby('Neighborhood').count() # how many venues were retruned for each borough

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Barking and Dagenham,14,14,14,14,14,14
Barnet,26,26,26,26,26,26
Bexley,9,9,9,9,9,9
Brent,68,68,68,68,68,68
Bromley,62,62,62,62,62,62
Camden,100,100,100,100,100,100
City of London,100,100,100,100,100,100
Croydon,88,88,88,88,88,88
Ealing,100,100,100,100,100,100


In [20]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(boroughs_venues['Venue Category'].unique())))

There are 255 uniques categories.


In [21]:
boroughs_venues[boroughs_venues['Venue Category'] == "Neighborhood"]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [22]:
# drop rows containing Venue Category = "Neighborhood", it has no meaning!
boroughs_venues = boroughs_venues[boroughs_venues['Venue Category'] != "Neighborhood"]

#### Now create one hot encoded data frame with the categories for each neighborhood

In [23]:
# one hot encoding
boroughs_onehot = pd.get_dummies(boroughs_venues[['Venue Category']], prefix="", prefix_sep="")

boroughs_onehot.head()

,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Caribbean Restaurant,Castle,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Imported Food Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Newsagent,Nightclub,Noodle House,Observatory,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waffle Shop,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [24]:
# Re-add Neighborhood column and make it the first column
columns_names = list(boroughs_onehot.columns.values)

boroughs_onehot['Neighborhood'] = boroughs_venues['Neighborhood']

columns_names.insert(0, 'Neighborhood')

boroughs_onehot = boroughs_onehot[columns_names]

boroughs_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Caribbean Restaurant,Castle,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Imported Food Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Newsagent,Nightclub,Noodle House,Observatory,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waffle Shop,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,City of London,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
boroughs_onehot[boroughs_onehot['Neighborhood'] == 'Barking and Dagenham']

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Caribbean Restaurant,Castle,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Imported Food Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Newsagent,Nightclub,Noodle House,Observatory,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waffle Shop,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
100,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [26]:
boroughs_onehot.shape

(2132, 256)

In [27]:
boroughs_grouped = boroughs_onehot.groupby('Neighborhood').mean().reset_index()
boroughs_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Caribbean Restaurant,Castle,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Imported Food Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Newsagent,Nightclub,Noodle House,Observatory,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waffle Shop,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Barking and Dagenham,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.071429,0.142857,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000

In [28]:
boroughs_grouped.shape

(33, 256)

In [29]:
def return_most_common_venues(row, c):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    #print(row_categories_sorted)

    sorted_categories_ar = row_categories_sorted.index.values[0:c]

    for i, value in enumerate(row_categories_sorted):
        if value == 0 and i < c:
            sorted_categories_ar[i] = ''
        elif i == c:
            break
    
    return sorted_categories_ar

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = boroughs_grouped['Neighborhood']

for ind in np.arange(boroughs_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boroughs_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Bus Stop,Supermarket,Gym,Bus Station,Pub,Convenience Store,Soccer Stadium,Gas Station,Park,Grocery Store
1,Barnet,Pub,Coffee Shop,Pizza Place,Pharmacy,Grocery Store,Bar,Newsagent,Soccer Stadium,Fast Food Restaurant,Shopping Mall
2,Bexley,Pub,Greek Restaurant,Fast Food Restaurant,Steakhouse,Museum,Breakfast Spot,Hotel,Italian Restaurant,,
3,Brent,Coffee Shop,Clothing Store,Sporting Goods Shop,Bar,Supermarket,Hotel,Café,Food Court,Pub,Italian Restaurant
4,Bromley,Pub,Clothing Store,Coffee Shop,Café,Hotel,Pizza Place,Supermarket,Bar,Gym / Fitness Center,Portuguese Restaurant


In [31]:
neighborhoods_venues_sorted.shape

(33, 11)

## 3.4. Cluster Neighborhoods

In [32]:
# set number of clusters
kclusters = 4

boroughs_grouped_clustering = boroughs_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boroughs_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2,
       0, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2])

In [33]:
kmeans.labels_.shape

(33,)

In [34]:
# add clustering labels
try:
    neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop("Cluster Labels", 1)
except:
    print("Cluster Labels not present")
    
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

Cluster Labels not present


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Barking and Dagenham,Bus Stop,Supermarket,Gym,Bus Station,Pub,Convenience Store,Soccer Stadium,Gas Station,Park,Grocery Store
1,2,Barnet,Pub,Coffee Shop,Pizza Place,Pharmacy,Grocery Store,Bar,Newsagent,Soccer Stadium,Fast Food Restaurant,Shopping Mall
2,3,Bexley,Pub,Greek Restaurant,Fast Food Restaurant,Steakhouse,Museum,Breakfast Spot,Hotel,Italian Restaurant,,
3,0,Brent,Coffee Shop,Clothing Store,Sporting Goods Shop,Bar,Supermarket,Hotel,Café,Food Court,Pub,Italian Restaurant
4,2,Bromley,Pub,Clothing Store,Coffee Shop,Café,Hotel,Pizza Place,Supermarket,Bar,Gym / Fitness Center,Portuguese Restaurant
5,2,Camden,Café,Pub,Coffee Shop,Zoo Exhibit,Bar,French Restaurant,Italian Restaurant,Bookstore,Japanese Restaurant,Park
6,2,City of London,Coffee Shop,Hotel,Gym / Fitness Center,Event Space,Cocktail Bar,Steakhouse,Italian Restaurant,French Restaurant,Scenic Lookout,Burger Joint
7,2,Croydon,Pub,Coffee Shop,Italian Restaurant,Clothing Store,Café,Grocery Store,Indian Restaurant,Park,Mediterranean Restaurant,Platform
8,2,Ealing,Coffee Shop,Pub,Italian Restaurant,Park,Bakery,Hotel,Pizza Place,Sandwich Place,Thai Restaurant,Burger Joint
9,2,Enfield,Pub,Coffee Shop,Clothing Store,Indian Restaurant,Park,Department Store,Turkish Restaurant,Supermarket,Pharmacy,Fish & Chips Shop


In [35]:
boroughs_merged = boroughs_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
boroughs_merged = neighborhoods_venues_sorted.join(boroughs_merged.set_index('Neighborhood'), on='Neighborhood')

boroughs_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
0,0,Barking and Dagenham,Bus Stop,Supermarket,Gym,Bus Station,Pub,Convenience Store,Soccer Stadium,Gas Station,Park,Grocery Store,E09000002,51.554091,0.150489
1,2,Barnet,Pub,Coffee Shop,Pizza Place,Pharmacy,Grocery Store,Bar,Newsagent,Soccer Stadium,Fast Food Restaurant,Shopping Mall,E09000003,51.653090,-0.200226
2,3,Bexley,Pub,Greek Restaurant,Fast Food Restaurant,Steakhouse,Museum,Breakfast Spot,Hotel,Italian Restaurant,,,E09000004,51.441679,0.150488
3,0,Brent,Coffee Shop,Clothing Store,Sporting Goods Shop,Bar,Supermarket,Hotel,Café,Food Court,Pub,Italian Restaurant,E09000005,51.563996,-0.275906
4,2,Bromley,Pub,Clothing Store,Coffee Shop,Café,Hotel,Pizza Place,Supermarket,Bar,Gym / Fitness Center,Portuguese Restaurant,E09000006,51.402805,0.014814


#### let's visualize the resulting clusters

In [36]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(boroughs_merged['Latitude'], boroughs_merged['Longitude'], boroughs_merged['Neighborhood'], boroughs_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print(label)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3.5. Examine Clusters

In [37]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 0, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
0,Barking and Dagenham,Bus Stop,Supermarket,Gym,Bus Station,Pub,Convenience Store,Soccer Stadium,Gas Station,Park,Grocery Store,E09000002,51.554091,0.150489
3,Brent,Coffee Shop,Clothing Store,Sporting Goods Shop,Bar,Supermarket,Hotel,Café,Food Court,Pub,Italian Restaurant,E09000005,51.563996,-0.275906
14,Harrow,Indian Restaurant,Thai Restaurant,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Park,Supermarket,Kitchen Supply Store,Gym,Metro Station,E09000015,51.596827,-0.337305
17,Hounslow,Indian Restaurant,Coffee Shop,Hotel,Fast Food Restaurant,Grocery Store,Clothing Store,Bus Stop,Gym / Fitness Center,Sandwich Place,Bakery,E09000018,51.468613,-0.361347
22,Lewisham,Supermarket,Pub,Gym,Clothing Store,Coffee Shop,Restaurant,Café,Bakery,Sandwich Place,Street Food Gathering,E09000023,51.462432,-0.010133
23,Merton,Coffee Shop,Sushi Restaurant,Supermarket,Clothing Store,Tram Station,Pub,Bar,Thai Restaurant,Lebanese Restaurant,Pizza Place,E09000024,51.410870,-0.188097


In [38]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 1, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
28,Sutton,Grocery Store,Italian Restaurant,Supermarket,Fish & Chips Shop,Park,Pub,,,,,E09000029,51.357464,-0.173627


In [39]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 2, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
1,Barnet,Pub,Coffee Shop,Pizza Place,Pharmacy,Grocery Store,Bar,Newsagent,Soccer Stadium,Fast Food Restaurant,Shopping Mall,E09000003,51.653090,-0.200226
4,Bromley,Pub,Clothing Store,Coffee Shop,Café,Hotel,Pizza Place,Supermarket,Bar,Gym / Fitness Center,Portuguese Restaurant,E09000006,51.402805,0.014814
5,Camden,Café,Pub,Coffee Shop,Zoo Exhibit,Bar,French Restaurant,Italian Restaurant,Bookstore,Japanese Restaurant,Park,E09000007,51.542797,-0.162480
6,City of London,Coffee Shop,Hotel,Gym / Fitness Center,Event Space,Cocktail Bar,Steakhouse,Italian Restaurant,French Restaurant,Scenic Lookout,Burger Joint,E09000001,51.515618,-0.091998
7,Croydon,Pub,Coffee Shop,Italian Restaurant,Clothing Store,Café,Grocery Store,Indian Restaurant,Park,Mediterranean Restaurant,Platform,E09000008,51.371305,-0.101957
8,Ealing,Coffee Shop,Pub,Italian Restaurant,Park,Bakery,Hotel,Pizza Place,Sandwich Place,Thai Restaurant,Burger Joint,E09000009,51.512655,-0.305195
9,Enfield,Pub,Coffee Shop,Clothing Store,Indian Restaurant,Park,Department Store,Turkish Restaurant,Supermarket,Pharmacy,Fish & Chips Shop,E09000010,51.652085,-0.081018
10,Greenwich,Pub,Garden,Grocery Store,Café,Coffee Shop,Bakery,Gym / Fitness Center,Park,Mediterranean Restaurant,Market,E09000011,51.482084,-0.004542
11,Hackney,Pub,Bakery,Coffee Shop,Café,Brewery,Vietnamese Restaurant,Cocktail Bar,Park,Gastropub,Yoga Studio,E09000012,51.543240,-0.049362
12,Hammersmith and Fulham,Pub,Coffee Shop,Hotel,Café,Thai Restaurant,Chinese Restaurant,Park,Gym / Fitness Center,Middle Eastern Restaurant,Sandwich Place,E09000013,51.498314,-0.227878


In [40]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 3, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
2,Bexley,Pub,Greek Restaurant,Fast Food Restaurant,Steakhouse,Museum,Breakfast Spot,Hotel,Italian Restaurant,,,E09000004,51.441679,0.150488


In [41]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 4, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [42]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 5, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [43]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 6, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [44]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 7, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [45]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 8, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [46]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 9, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [47]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 10, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [48]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 11, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
